# Using regex to fix dataset irregularities with jockey and trainer names

## Original dataset was likely merged together from multiple sources, as the jockey and trainer names are written differently throughout.

## This was done in order to incorporate other datasets that have jockey and trainer win percentage information.

## It was ultimately unsuccessful because the jockey and trainer win percentage datasets only had information for the "Top 100" for each year. This was not enough information to sufficiently fill out the jockey and trainer information for a substantial number of races.

## Still, this endeavor helped me practice some of my regex and automation skills.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chisquare, ttest_ind, zscore

import re
import itertools

%matplotlib inline

#Supresses scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import warnings
warnings.filterwarnings('ignore')

In [2]:
#There are a lot of issues with the 'position_two' column, so I left it out.
#Furthermore, the 'position_again' column is much more consistent and has all relevant win/place information  

fields = ["position_again","bf_odds","venue_name","date","market_name","condition",
          "barrier","handicap_weight","last_five_starts","prize_money","sex",
          "age","jockey_sex","days_since_last_run","overall_starts","overall_wins",
          "overall_places","track_starts","track_wins","track_places","firm_starts",
          "firm_wins","firm_places","good_starts","good_wins","good_places",
          "slow_starts","slow_wins","slow_places","soft_starts","soft_wins",
          "soft_places","heavy_starts","heavy_wins","heavy_places","distance_starts",
          "distance_wins","distance_places","jockey","trainer",]

df = pd.read_csv("horses.csv", skipinitialspace=True, usecols=fields, low_memory=False)

df.head()

,position_again,bf_odds,venue_name,date,market_name,condition,barrier,handicap_weight,last_five_starts,prize_money,...,slow_places,soft_starts,soft_wins,soft_places,heavy_starts,heavy_wins,heavy_places,distance_starts,distance_wins,distance_places
0,1.00,2.88,Echuca,2016-06-27,R3 1200m Mdn,HVY9,3.00,58.50,f3,2160.00,...,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
1,2.00,15.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,10.00,58.50,x80x2,21175.00,...,1.00,2.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00
2,3.00,95.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,6.00,56.50,79x00,28855.00,...,0.00,5.00,0.00,0.00,3.00,0.00,0.00,4.00,0.00,0.00
3,nan,20.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,2.00,56.50,f0,1475.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,nan,2.74,Echuca,2016-06-27,R3 1200m Mdn,HVY9,7.00,56.50,f4x,1215.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [3]:
df.date = pd.to_datetime(df.date, format='%Y'+'-'+'%m'+'-'+'%d')

#removes numbers from end of 'condition' strings
df.condition = df.condition.str.replace('\d+', '')

#renaming condition values so that they're uniform
df.condition = df.condition.replace(['HVY','AWT'], ['HEAVY','GOOD']) 
#AWT equates to a Good surface under some weather conditions

#reverses 'last_five_starts' (originally written right-to-left) 
#so that it's easier to read in the future
df.last_five_starts = df.last_five_starts.str[::-1]

## New code starts here:

In [4]:
df_trainer_15_16 = pd.read_csv("punters-com-au-downloaded-stats-trainers-2019-06-29 07_02_58.csv")
df_trainer_15_16.drop(['Firsts', 'Seconds', 'Thirds', 'ROI'],axis=1,inplace=True)
df_trainer_15_16.Trainers = df_trainer_15_16.Trainers.str.upper()

In [5]:
df_trainer_16_17 = pd.read_csv("punters-com-au-downloaded-stats-trainers-2019-06-29 07_03_10.csv")
df_trainer_16_17.drop(['Firsts', 'Seconds', 'Thirds', 'ROI'],axis=1,inplace=True)
df_trainer_16_17.Trainers = df_trainer_16_17.Trainers.str.upper()

In [6]:
df_trainer_17_18 = pd.read_csv("punters-com-au-downloaded-stats-trainers-2019-06-29 07_03_22.csv")
df_trainer_17_18.drop(['Firsts', 'Seconds', 'Thirds', 'ROI'],axis=1,inplace=True)
df_trainer_17_18.Trainers = df_trainer_17_18.Trainers.str.upper()

## The code below first attempts to make the jockey/trainer names in the original dataset more uniform, as they were abbreviated irregularly throughout.

## It then compares those names to the win percentage datasets, and if there is no ambiguity, it converts the abbreviated names to the win percentage names. 

## It also prints out examples where there is ambiguity so that I can replace it manually.

### The output below are the ambiguous and more difficult fixed cases:

In [7]:
last_name = list(set(df[df.trainer.str.match("^[a-zA-Z]{1} ",str,na=False)].trainer.str.split().str.get(-1)))
fix_names = []
first_initial_error = []

for name in last_name:
    trainer_data_15_16 = df_trainer_15_16[df_trainer_15_16.Trainers.str.contains(r"\b"+name+"$",str,na=False)].Trainers
    trainer_data_16_17 = df_trainer_15_16[df_trainer_16_17.Trainers.str.contains(r"\b"+name+"$",str,na=False)].Trainers
    trainer_data_17_18 = df_trainer_15_16[df_trainer_17_18.Trainers.str.contains(r"\b"+name+"$",str,na=False)].Trainers

    if (len(trainer_data_15_16)>0)|(len(trainer_data_16_17)>0)|(len(trainer_data_17_18)>0):
        pass
    else:
        continue
    
    name_pair = list(set(df[df.trainer.str.contains(r"\b"+name+"$",str,na=False)].trainer))
    
    if len(name_pair)==2:
        split_list = [name_pair[0].split(),name_pair[1].split()]
        
        if len(split_list[0][0])<len(split_list[1][0]):
            split_list[0],split_list[1] = split_list[1],split_list[0]
        elif len(split_list[0][0])==len(split_list[1][0]):
            print("what is this:",split_list[0][0])
            continue
        
        if (list(split_list[0][0])[0]!=split_list[1][0])|(len(split_list[1][0])>1):
            first_initial_error.append(split_list)
            continue
        
        list1 = list(trainer_data_15_16.str.split().str.get(0))
        list2 = list(trainer_data_16_17.str.split().str.get(0))
        list3 = list(trainer_data_17_18.str.split().str.get(0))

        tot_list = list(set(list1 + list2 + list3))
        
        if any(split_list[0][0] == x for x in tot_list):
            full_name = " ".join(split_list[0])
            abbrev_name = " ".join(split_list[1])

            df.trainer = df.trainer.str.replace(full_name,abbrev_name)
            #Avoids repetitive strings that are difficult to account for:
            df.trainer = df.trainer.str.replace(abbrev_name,full_name)

            fix_names.append([full_name,abbrev_name])
        else:
            continue   
    elif len(name_pair)>2:
        split_list1 = []
        for x in name_pair:
            split_list1.append(x.split())
        
        list1 = list(trainer_data_15_16.str.split().str.get(0))
        list2 = list(trainer_data_16_17.str.split().str.get(0))
        list3 = list(trainer_data_17_18.str.split().str.get(0))

        tot_list = list(set(list1 + list2 + list3))
        
        if any(a == b[0] for a in tot_list for b in split_list1):
            for a in tot_list:
                for b in split_list1:
                    if a == b[0]:
                        first_initial = list(a)[0]
                        initialized_name = list(set(df[df.trainer.str.match(r"\b"+first_initial+"{1} .*"+name+"$",str,na=False)].trainer))
                        if len(initialized_name)!=1:
                            if len(initialized_name)>=2:
                                print(initialized_name)
                                print("names to check:",a,name,"\n"*2)
                            continue
                        else:
                            full_name1 = a+" "+name
                            print(initialized_name)
                            if len(initialized_name[0].split())>3:
                                print("problem with fix:", initialized_name[0],"\n")
                                continue
                            else:
                                print("fixed:",full_name1,', ',initialized_name[0],"\n")
                                df.trainer = df.trainer.str.replace(full_name1,initialized_name[0])
                                #Avoids repetitive strings that are difficult to account for:
                                df.trainer = df.trainer.str.replace(initialized_name[0],full_name1)                  

['C R SCOBY-SMITH', 'C SMITH']
names to check: CRAIG SMITH 


['B J SMITH']
fixed: BARRY SMITH ,  B J SMITH 

['L SMITH', 'L J MAYFIELD SMITH']
names to check: LINDSEY SMITH 


['M W SMITH', 'M K SMITH', 'M J SMITH']
names to check: MATTHEW SMITH 


['B C BAKER']
fixed: BJORN BAKER ,  B C BAKER 

['L ROSS']
fixed: LES ROSS ,  L ROSS 

['D T KELLY']
fixed: DAVID KELLY ,  D T KELLY 

['N J OLIVE']
fixed: NICK OLIVE ,  N J OLIVE 

['D E OXLADE']
fixed: DARRYL OXLADE ,  D E OXLADE 

['D L CASEY']
fixed: DAVE CASEY ,  D L CASEY 

['S P CASEY', 'S R CASEY']
names to check: SEAN CASEY 


['P J CUNNINGHAM']
fixed: PADDY CUNNINGHAM ,  P J CUNNINGHAM 

['D R PATEMAN']
fixed: DARRYN PATEMAN ,  D R PATEMAN 

['J P THOMPSON']
fixed: JOHN THOMPSON ,  J P THOMPSON 

['K M SWEENEY']
fixed: KEN SWEENEY ,  K M SWEENEY 

['M A KAVANAGH']
fixed: MARK KAVANAGH ,  M A KAVANAGH 

['G K WHITE', 'G S WHITE']
names to check: GARY WHITE 


['J N ENGLEBRECHT']
fixed: JEFF ENGLEBRECHT ,  J N ENGLEBRECHT 

['G R DU

### Manually replacing the determinable ambiguous cases:

In [8]:
df.trainer = df.trainer.replace(['G & A WILLIAMS','M J WILLIAMS','D A WILLIAMS','MS G WILLIAMS','MS S WILLIAMS'],
                                ['GRANT & ALANA WILLIAMS','MATTHEW WILLIAMS','DANNY WILLIAMS','GAYNA WILLIAMS','SHYLIE WILLIAMS'])
df.trainer = df.trainer.str.replace('W F FRANCIS & G KENT','WAYNE FRANCIS & GLEN KENT')
df.trainer = df.trainer.str.replace('D & B PEARCE','DANIEL & BEN PEARCE')
df.trainer = df.trainer.str.replace('D & B HAYES & T DABERNIG','DAVID & B HAYES & T DABERNIG')
df.trainer = df.trainer.replace(['R & L PRICE','R W PRICE'],
                                ['ROBERT & LUKE PRICE','ROSS PRICE'])
df.trainer = df.trainer.str.replace('G & A WILLIAMS','GRANT & ALANA WILLIAMS')
df.trainer = df.trainer.str.replace('P H DUNCAN','PAUL H DUNCAN')
df.trainer = df.trainer.str.replace('B & D GUY','BRYAN & DANIEL GUY')
df.trainer = df.trainer.replace(['T M ANDREWS','T J ANDREWS'],
                                ['TREVOR ANDREWS','TREVOR J ANDREWS'])
#This is an issue that I noticed with duplicated names.
#I used this workaround to have it match the trainer datasets below 
df.trainer = df.trainer.str.replace('DAVID & B HAYES & T DABERNIG','D & B HAYES & T DABERNIG')
df.trainer = df.trainer.str.replace('D & B HAYES & T DABERNIG','DAVID & B HAYES & T DABERNIG')

### The names that were automatically fixed without difficulty:

In [9]:
fix_names

[['ERIC MUSGROVE', 'E V MUSGROVE'],
 ['BRETT BELLAMY', 'B D BELLAMY'],
 ['STUART KENDRICK', 'S W KENDRICK'],
 ['MICHAEL MORONEY', 'M D MORONEY'],
 ['JEREMY SYLVESTER', 'J A SYLVESTER'],
 ['ASHLEY MALEY', 'A W MALEY'],
 ['JARED WEHLOW', 'J WEHLOW'],
 ['ROBBIE LAING', 'R E LAING'],
 ['STEVE WOLFE', 'S J WOLFE'],
 ['NEVILLE PARNHAM', 'N D PARNHAM'],
 ['JOHN SARGENT', 'J G SARGENT'],
 ['FRED WIELAND', 'F J WIELAND'],
 ['PETER & PAUL SNOWDEN', 'P & P SNOWDEN'],
 ['BRETT CAVANOUGH', 'B F CAVANOUGH'],
 ['GAI WATERHOUSE & ADRIAN BOTT', 'G WATERHOUSE & A BOTT'],
 ['JASON COYLE', 'J C COYLE'],
 ['JOHN SADLER', 'J D SADLER'],
 ['JOHN ZIELKE', 'J W ZIELKE'],
 ['WAYNE WALTERS', 'W H WALTERS'],
 ['MICK MAIR', 'M J MAIR'],
 ['TREVOR SUTHERLAND', 'T G SUTHERLAND'],
 ['KEVIN KEMP', 'K R KEMP'],
 ['REX LIPP', 'R G LIPP'],
 ['GARY PORTELLI', 'G PORTELLI'],
 ['TOM BUTTON', 'T BUTTON'],
 ['PAUL FACOORY', 'P J FACOORY'],
 ['MATHEW ELLERTON & SIMON ZAHRA', 'M ELLERTON & S ZAHRA'],
 ['RODNEY NORTHAM', 'R P NO

### These were irregular names, where the first initial of the abbreviated name did not match the first letter of the win percentage full name but where there was no other individual with that last name:

In [10]:
first_initial_error

[[['NEIL', 'GODBOLT'], ['R', 'N', 'GODBOLT']],
 [['ROY', 'CHILLEMI'], ['O', 'CHILLEMI']],
 [['ROSS', 'STITT'], ['M', 'R', 'STITT']],
 [['BOB', 'MILLIGAN'], ['R', 'R', 'MILLIGAN']]]

In [11]:
df.trainer = df.trainer.str.replace('M R STITT','ROSS STITT')
df.trainer = df.trainer.str.replace('R N GODBOLT','NEIL GODBOLT')
df.trainer = df.trainer.str.replace('R R MILLIGAN','BOB MILLIGAN')
df.trainer = df.trainer.str.replace('O CHILLEMI','ROY CHILLEMI')

### Doing the same for the jockey win percentage datasets.

### Jockies names were easier to change, as some trainers work in teams with combined names.

In [12]:
df_jockey_15_16 = pd.read_csv("punters-com-au-downloaded-stats-jockeys-2019-06-29 07_02_19.csv")
df_jockey_15_16.drop(['Firsts', 'Seconds', 'Thirds', 'ROI'],axis=1,inplace=True)
df_jockey_15_16.Jockeys = df_jockey_15_16.Jockeys.str.upper()

In [13]:
df_jockey_16_17 = pd.read_csv("punters-com-au-downloaded-stats-jockeys-2019-06-29 07_02_32.csv")
df_jockey_16_17.drop(['Firsts', 'Seconds', 'Thirds', 'ROI'],axis=1,inplace=True)
df_jockey_16_17.Jockeys = df_jockey_16_17.Jockeys.str.upper()

In [14]:
df_jockey_17_18 = pd.read_csv("punters-com-au-downloaded-stats-jockeys-2019-06-29 07_02_39.csv")
df_jockey_17_18.drop(['Firsts', 'Seconds', 'Thirds', 'ROI'],axis=1,inplace=True)
df_jockey_17_18.Jockeys = df_jockey_17_18.Jockeys.str.upper()

In [15]:
original_abbrev = df.jockey[df.jockey!="Not declared"].value_counts().index
fixed_list = []
for bad_name in original_abbrev:
    name = re.sub(r"[^\w\s]",' ',bad_name).split()
    
    jockey_data_15_16 = df_jockey_15_16[df_jockey_15_16.Jockeys.str.contains(r"\b"+name[-1]+"$",str,na=False)].Jockeys
    jockey_data_16_17 = df_jockey_16_17[df_jockey_16_17.Jockeys.str.contains(r"\b"+name[-1]+"$",str,na=False)].Jockeys
    jockey_data_17_18 = df_jockey_17_18[df_jockey_17_18.Jockeys.str.contains(r"\b"+name[-1]+"$",str,na=False)].Jockeys

    if (len(jockey_data_15_16)>0)|(len(jockey_data_16_17)>0)|(len(jockey_data_17_18)>0):
        pass
    else:
        continue
    
    list1 = list(jockey_data_15_16.str.split())
    list2 = list(jockey_data_16_17.str.split())
    list3 = list(jockey_data_17_18.str.split())

    tot_list = list1 + list2 + list3
    tot_list.sort()

    tot_list = list(i for i,_ in itertools.groupby(tot_list))
    
    if any(list(a[0])[0]==name[0] for a in tot_list):
        good_list = []
        for x in tot_list:
            if list(x[0])[0]==name[0]:
                good_list.append(" ".join(x))
        if len(good_list)==1:
            fixed_list.append(good_list[0])
            
            df.jockey = df.jockey.str.replace(bad_name,good_list[0])              
        else:
            print("NEED TO FIX:")
            print("good_list:",good_list)
            print("test list:"," ".join(name))
            print(bad_name,'\n'*2)

NEED TO FIX:
good_list: ['BEN E THOMPSON', 'BEN THOMPSON']
test list: B THOMPSON
B.THOMPSON 


NEED TO FIX:
good_list: ['MATHEW CAHILL', 'MICHAEL CAHILL']
test list: M J CAHILL
M.J.CAHILL 


NEED TO FIX:
good_list: ['CHRIS WILLIAMS', 'CRAIG WILLIAMS']
test list: C WILLIAMS
C.WILLIAMS 


NEED TO FIX:
good_list: ['MATHEW CAHILL', 'MICHAEL CAHILL']
test list: M A CAHILL
M.A.CAHILL 


NEED TO FIX:
good_list: ['DWAYNE DUNN', 'DYLAN DUNN']
test list: D DUNN
D.DUNN 


NEED TO FIX:
good_list: ['JASON TAYLOR', 'JESS TAYLOR']
test list: J L TAYLOR
J.L.TAYLOR 


NEED TO FIX:
good_list: ['CEJAY GRAHAM', 'CHRIS GRAHAM']
test list: C GRAHAM
C.GRAHAM 


NEED TO FIX:
good_list: ['JASON TAYLOR', 'JESS TAYLOR']
test list: J TAYLOR
J.TAYLOR 


NEED TO FIX:
good_list: ['JARRAD NOSKE', 'JERRY NOSKE']
test list: J J NOSKE
J.J.NOSKE 


NEED TO FIX:
good_list: ['JUSTIN P STANLEY', 'JUSTIN STANLEY']
test list: J P STANLEY
J.P.STANLEY 


NEED TO FIX:
good_list: ['JARRAD NOSKE', 'JERRY NOSKE']
test list: J NOSKE

In [16]:
df.jockey = df.jockey.str.replace('B.THOMPSON','BEN E THOMPSON')
df.jockey = df.jockey.str.replace('B.THOMPSON','BEN THOMPSON')
df.jockey = df.jockey.str.replace('M.A.CAHILL','MATHEW CAHILL')
df.jockey = df.jockey.str.replace('M.J.CAHILL','MICHAEL CAHILL')
df.jockey = df.jockey.str.replace('J.L.TAYLOR','JASON TAYLOR')
df.jockey = df.jockey.str.replace('J.TAYLOR','JESS TAYLOR')
df.jockey = df.jockey.str.replace('J.J.NOSKE','JERRY NOSKE')
df.jockey = df.jockey.str.replace('J.NOSKE','JARRAD NOSKE')
df.jockey = df.jockey.str.replace('J.P.STANLEY','JUSTIN P STANLEY')
df.jockey = df.jockey.str.replace('C.C WILLIAMS','CHRIS WILLIAMS')
df.jockey = df.jockey.str.replace('C.WILLIAMS','CRAIG WILLIAMS')
df.jockey = df.jockey.str.replace('D.J DUNN','DYLAN DUNN')
df.jockey = df.jockey.str.replace('D.DUNN','DWAYNE DUNN')
df.jockey = df.jockey.str.replace('M.A.CAHILL *','MATHEW CAHILL')
df.jockey = df.jockey.str.replace('M.A.CAHILL','MATHEW CAHILL')
df.jockey = df.jockey.str.replace('M.J.CAHILL','MICHAEL CAHILL')
df.jockey = df.jockey.str.replace('J.TAYLOR *','JESS TAYLOR')
df.jockey = df.jockey.str.replace('J.L.TAYLOR *','JASON TAYLOR')
df.jockey = df.jockey.str.replace('M.J.CAHILL *','MICHAEL CAHILL')
df.jockey = df.jockey.str.replace('J.J.NOSKE *','JERRY NOSKE')
df.jockey = df.jockey.str.replace('J.P.STANLEY *','JUSTIN P STANLEY')
df.jockey = df.jockey.str.replace('J.NOSKE *','JARRAD NOSKE')
df.jockey = df.jockey.str.replace('C.C WILLIAMS *','CHRIS WILLIAMS')
df.jockey = df.jockey.str.replace('D.J DUNN *','DYLAN DUNN')

In [17]:
fixed_list

['JEFF PENZA',
 'GRANT BUCKLEY',
 'BRENTON AVDULLA',
 'TIM CLARK',
 'JAY FORD',
 'JIM BYRNE',
 'DEAN YENDALL',
 'DAMIEN THORNTON',
 'TYE ANGLAND',
 'JAMIE MOTT',
 'JARROD FRY',
 'BEN ALLEN',
 'BEN LOOKER',
 'GREG RYAN',
 'JEFF LLOYD',
 'MICHEAL HELLYER',
 'RACHAEL MURRAY',
 'DANIEL MOOR',
 'MICHAEL DEE',
 'JASON COLLETT',
 'JEAN VAN OVERMEIRE',
 'JORDAN CHILDS',
 'WILLIAM PIKE',
 'LINDA MEECH',
 'DAMIAN LANE',
 'CRAIG NEWITT',
 'HARRY COFFEY',
 'TROY TURNER',
 'BRAD RAWILLER',
 'STEVEN PARNHAM',
 'JAMES ORMAN',
 'DANIEL STACKHOUSE',
 'SAIRYN FAWKE',
 'MATTHEW PAGET',
 'COREY BROWN',
 'JOHN ALLEN',
 'ANDREW GIBBONS',
 'PETER GRAHAM',
 'JAMIE KAH',
 'BLAKE SHINN',
 'RACHEL KING',
 'ROBBIE FRADD',
 'BELINDA HODDER',
 'BEAU MERTENS',
 'BRIAN HIGGINS',
 'CRAIG ROBERTSON',
 'ANDREW ADKINS',
 'NOEL CALLOW',
 'PETER KNUCKEY',
 'RANDY TAN',
 'GEORGINA CARTWRIGHT',
 'CLINT JOHNSTON-PORTER',
 'JACK HILL',
 'CHRISTIAN REITH',
 'JUSTIN POTTER',
 'MITCHELL BELL',
 'JOSEPH AZZOPARDI',
 'JASON WHITING